# BBH Detection with BBHNet

What happens when the signal from a BBH event starts to enter BBHNet's input kernel?

In [1]:
import os

import h5py
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

from analysis import OUTPUT_DIR

output_notebook()

event_tc = [1186302519.8, 1186741861.5, 1187058327.1, 1187529256.5]
event_names = ['GW170809', 'GW170814', 'GW170818', 'GW170823']

Loading BokehJS ...

In [2]:
def read_fname(fname):
    with h5py.File(fname, "r") as f:
        # add a 1 to index by the _end_ of the kernel
        t = f["GPSstart"][:] + 1
        y = f["out"][:, 0]
    return t, y

event_fname = os.path.join(
    OUTPUT_DIR, "dt-0.0", "5", "out", "out_1186301952-1024.hdf5"
)
t, y = read_fname(event_fname)
t = t - event_tc[0]

# zoom in right around event
mask = (-1 <= t) & (t <= 2)
t = t[mask]
y = y[mask]

In [3]:
def plot_step(up_until: float, show_mf: bool = False):
    p = figure(
        height=400,
        width=800,
        x_range=(-1, 2),
        x_axis_label="Time from signal entering kernel (s)",
        y_range=(y.min() - 1, y.max() + 1),
        tools=""
    )

    mask = t <= up_until
    p.line(t[mask], y[mask])

    if show_mf:
        p.line(
            [-0.5, -0.1, -0.1, 0.9, 0.9, 1.3],
            [-7, -7, 13, 13, -7, -7],
            line_width=2.3,
            line_color="#000000"
        )
    show(p)

In [4]:
plot_step(-0.1)

In [5]:
plot_step(0.01)

In [6]:
plot_step(0.1)

In [7]:
plot_step(0.25)

In [8]:
plot_step(0.5)

In [9]:
plot_step(1)

In [10]:
plot_step(1.5)

What if we match filtered this canonical event shape in NN-output space?

In [11]:
p = plot_step(1.5, show_mf=True)

Equivalently: average of last 1s of NN outputs.

In [12]:
import numpy as np
from scipy.signal import convolve

def match_filter(t, y, window_length: float = 1.0):
    sample_rate = 1 / (t[1] - t[0])

    window_size = int(window_length * sample_rate)
    window = np.ones((window_size,)) / window_size

    mf = convolve(y, window, mode="valid")
    t = t[window_size - 1:]  # slice off time samples to make y average of the past
    return t, mf

In [13]:
from bokeh.models import (
    BoxZoomTool,
    ColumnDataSource,
    HoverTool,
    LinearAxis,
    Range1d
)
    
def plot_time_series(t, y, mf):
    """Quick function to plot event outputs and mf responses"""

    source = ColumnDataSource(dict(t=t, y=mf, nn=y))
    p = figure(
        title=f"Locally averaged NN outputs",
        height=300,
        width=900,
        x_axis_label="Time from Event (s)",
        y_axis_label="Matched Filter Output",
        tools="reset"
    )

    # add a y-range on the right for the raw nn outputs
    p.extra_y_ranges = {"nn": Range1d(1.01*y.min(), 1.01*y.max())}
    p.add_layout(LinearAxis(y_range_name="nn", axis_label="NN output"), "right")

    # plot the two time series on the different y-axes
    r = p.line(x="t", y="y", legend_label="MF output", source=source)
    p.line(
        x="t",
        y="nn",
        line_color="#000000",
        legend_label="Raw NN output",
        y_range_name="nn",
        line_alpha=0.6,
        line_dash="2 2",
        source=source
    )

    # add a couple tools for playing with the data
    p.add_tools(
        HoverTool(
            mode="vline",
            tooltips=[("Time from Event", "@t s"), ("Local Average", "@y")],
            renderers=[r],
        ),
        BoxZoomTool(
            dimensions="width"
        )
    )

    # make the legend interactive
    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    show(p)

Let's run this filter on the event from earlier

In [14]:
t, y = read_fname(event_fname)
t, mf = match_filter(t, y)
plot_time_series(t - event_tc[0], y[-len(t):], mf)

Varying our cutoff threshold trades off _latency_ required to make a detection (i.e. achieve a certain level of sensitivity)

In [15]:
mask = np.abs((t - event_tc[0])) < 2
plot_time_series((t - event_tc[0])[mask], y[-len(t):][mask], mf[mask])

## Building a background
Taking a cue from [GW150914: First results from the search for binary black hole coalescence with Advanced LIGO](https://dcc.ligo.org/public/0123/P1500269/016/LIGO-P1500269_GW150914_CBC_Search.pdf)
- Use time shifted data from _before_ the first event in our mini-catalouge as background data
- Compute matched filter output distribution
- Evaluate significance of events in 0-shift data relative to this distribution

For simplicity here:
- Initialize bins up front
- Only use equivalent of 1 year of data

In [16]:
from typing import Optional, Tuple
from rich.progress import Progress
from analysis import parallel_search

def build_background(
    min_value: float =-5,
    max_value: float = 5,
    num_bins: int = 1000,
    num_proc: int = 4,
    max_tb: Optional[float] = None,
    norm_seconds: Optional[float] = None
) -> Tuple[np.ndarray, np.ndarray, float]:
    """
    Build a distribution of matched filter outputs on
    the first 5 runs of each time-shifted dataset, or
    until `max_tb` seconds of data have been processed.

    Args:
        min_value:
            The minimum value of the bins to use for
            histogramming matched filter outputs
        max_value:
            The maximum value of the bins to use
            for histogramming matched filter outputs
        num_bins:
            The number of bins to use for histogramming
            matched filter outputs
        num_proc:
            The number of parallel processes to use for
            computing and histogramming the matched
            filter outputs
        max_tb:
            The maximum number of equivalent seconds of
            data to process before stopping. If left as
            `None`, all the time shifted data will be
            processed.
        norm_seconds:
            The number of seconds of neural network
            outputs before each filter window
            to use to whiten the filter response
    Returns:
        The count of matched filter responses in each bin
        The center of each bin
        The total number of seconds processed
    """
    # initialize our histogram and time counter up front
    Tb = 0
    bins = np.linspace(min_value, max_value, num_bins + 1)
    counts = np.zeros((num_bins,))

    # do everything with a progress bar so we know
    # how long we'll be sitting here
    with Progress() as progress:
        # if we indicated a max amount of time to process,
        # use this for the progress bar. Otherwise just
        # approximate the number of runs we'll need to process
        if max_tb is not None:
            total = max_tb
        else:
            total = (len(shift_dirs) - 1) * 5
        task_id = progress.add_task("Computing background", total=total)

        # ignore the 0 shift data
        shifts = [i for i in os.listdir(OUTPUT_DIR) if i != "dt-0.0"]

        # only process the first 4 runs
        runs = range(5)

        # now compute the matched filter outputs in
        # parallel then histogram them as they
        # come back in
        for t, _, s in parallel_search(
            num_proc, shifts, runs, norm_seconds=norm_seconds
        ):
            Tb += t[-1] - t[0]
            counts += np.histogram(s, bins)[0]
            if max_tb is not None:
                progress.update(task_id, completed=Tb)
                if Tb >= max_tb:
                    break
            else:
                progress.update(task_id, advance=1)

    bin_centers = (bins[:-1] + bins[1:]) / 2
    return counts, bin_centers, Tb

In [17]:
max_tb = 3600 * 24 * 365
counts, bin_centers, Tb = build_background(
    min_value=-50, max_value=50, num_bins=1000, num_proc=8, max_tb=max_tb
)

Output()

In [18]:
from bokeh.models import LogAxis
from bokeh.palettes import Colorblind8 as palette

def plot_background(counts, bins, Tb):
    # since we want to log the y-axis, set a minimum
    # value that's greater than 0
    min_value = 0.5 * counts[counts > 0].min()
    p = figure(
        height=300,
        width=900,
        tools="reset",
        title=f"Background distribution from {Tb:0.1f}s of data",
        x_axis_label="Matched Filter Output",
        y_axis_label="Count",
        y_axis_type="log",
        y_range=(min_value, 1.05 * counts.max()),
        x_range=(bins[0], bins[-1]),
    )

    fars = np.cumsum(counts[::-1])
    fars = fars[::-1] / Tb
    fars = fars * 365 * 3600 * 24
    min_far = 0.5 * fars[fars > 0].min()

    # add a y-range on the right for the false alarm rate
    p.extra_y_ranges = {"far": Range1d(min_far, 1.5*fars.max())}
    p.add_layout(
        LogAxis(y_range_name="far", axis_label="Fals Alarm Rate (yr^-1)"),
        "right"
    )

    source = ColumnDataSource(dict(
        x=bins,
        top=np.clip(counts, min_value, np.inf),
        far=fars,
        values=counts # include the raw values for hover inspectino
    ))

    r = p.vbar(
        "x",
        top="top",
        bottom=min_value,
        width=0.95*(bins[1] - bins[0]),
        source=source,
        line_width=0.1,
        fill_alpha=0.6,
        fill_color=palette[0],
        line_color=palette[0],
        legend_label="Count"
    )

    # plot the false alarm rate on an interpolated
    # line to reduce the amount of memory required
    # it's pretty smooth so this won't change things
    # too much, plus we have the "real" values in the
    # data source anyway
    max_bin = bins[fars == 0].min() + 5 * (bins[1] - bins[0])
    far_interp_x = np.linspace(bins[0], max_bin, 200)
    far_interp = np.interp(far_interp_x, bins, fars)

    p.line(
        far_interp_x,
        far_interp,
        line_width=2.3,
        line_color=palette[1],
        legend_label="False Alarm Rate (yr^-1)",
        y_range_name="far"
    )

    # add a couple tools for playing with the data
    p.add_tools(
        HoverTool(
            mode="vline",
            tooltips=[("Bin Center", "@x"), ("Count", "@values"), ("FAR", "@far yr^-1")],
            renderers=[r],
        ),
        BoxZoomTool(
            dimensions="width"
        )
    )

    # make the legend interactive
    p.legend.click_policy = "hide"
    show(p)

Let's plot this background distribution, with an interpolated false alarm rate in units of $\text{yr}^{-1}$ on a second axis

In [19]:
plot_background(counts, bin_centers, Tb)

### <span style="color:#4287f5">AI #1: Widen and analyze background distribution</span>
- Build tool that, for a selected range of MF outputs, shows where in time the corresponding samples came from, which time slice they came from, and be able to inspect Q-scans of individual samples
- Keep raw filter outputs to avoid approximation errors introduced by binning
- Do analysis in Fig. 5 in GW150914 paper to make sure time slices are truly independent

# What can we detect with this?
Start by looking at the handful of samples we have.

In [20]:
def analyze_events(num_proc=8, norm_seconds=None):
    T = 0
    outputs = {}
    for t, y, s in parallel_search(
        num_proc, shifts=["dt-0.0"], runs=range(5, 16), norm_seconds=norm_seconds
    ):
        T += t[-1] - t[0]
        for tc, event_name in zip(event_tc, event_names):
            if event_name in outputs:
                continue
            elif t[0] <= tc <= t[-1]:
                t = t - tc
                mask = np.abs(t) < 60
                outputs[event_name] = (t[mask], y[mask], s[mask])
                break
        if len(outputs) == len(event_names):
            break
    return T, outputs

In [21]:
T, events = analyze_events()

In [22]:
plot_time_series(*events[event_names[0]])

In [23]:
plot_time_series(*events[event_names[1]])

In [24]:
plot_time_series(*events[event_names[2]])

In [25]:
plot_time_series(*events[event_names[3]])

Varying degrees of signal quality
- For each threshold during signal ascent, what would the corresponding FAR be?
- What is the latency we would incur to achieve that FAR?

In [26]:
import warnings
from scipy.special import erfinv

cmap = {event: color for event, color in zip(event_names, palette)}

def plot_far_vs_latency(events, counts, bins, Tb):
    p = figure(
        height=400,
        width=600,
        y_axis_type="log",
        y_axis_label="False Alarm Rate (yr^-1)",
        x_axis_label="Time from event entering kernel (s)",
        tools=""
    )

    source = ColumnDataSource()
    for event_name, (t, y, mf) in events.items():
        idx = np.argmin(np.abs(t))
        window = mf[idx: idx + 16 * 2]
        max_idx = np.argmax(window)
        window = window[:max_idx]

        fars, times = [], []
        for i, threshold in enumerate(window):
            count = counts[bins >= threshold].sum()
            far = 365 * 3600 * 24 * count / Tb
            # far = gammaincc(2, count * T / Tb)
            fars.append(max(far, 1e-6))
            times.append(t[idx + i])

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            source.data[f"t_{event_name}"] = times
            source.data[f"far_{event_name}"] = fars
        r = p.line(
            f"t_{event_name}",
            f"far_{event_name}",
            line_width=2.3,
            line_color=cmap[event_name],
            legend_label=event_name,
            source=source
        )

    p.add_tools(
        HoverTool(
            renderers=[r],
            tooltips=[
                (f"{event_name} FAR", f"@far_{event_name}")
                for event_name in events.keys()
            ],
            mode="vline",
        )
    )
    show(p)

In [27]:
plot_far_vs_latency(events, counts, bin_centers, Tb)

Not great right now, but this is only a handful of data points
- Couldn't draw conclusions for _any_ detection method using this, even if it was good

### <span style="color:#4287f5">AI #2: Use injections on background data to get signal distribution</span>
### <span style="color:#4287f5">AI #3: Use signal distribution to draw FAR vs. TPR for incremental average latency levels</span>

### <span style="color:#4287f5">AI #4: Optimize detection method using FAR vs. TPR vs. Latency curves</span>
- E.g. fix target TPR and optimize FAR within maximum latency

###  <span style="color:#4287f5">AI #5: Test optimized method on known O2 events</span>
- Some sort of significance metric for detections of known events
- E.g. relative likelihood: $\Lambda = \log \frac{p(H_1|\rho)}{p(H_0|\rho)}$
- Exponential distribution: $\lambda = \hat{n}_{H_0}(\rho)\frac{T}{T_b} \Rightarrow \Gamma(2,\hat{n}_{H_0}(\rho)\frac{T}{T_b})$

###  <span style="color:#4287f5">AI #6: Run optimized and tested method on O3 data to detect new event</span>

## Next approach
Normalize matched filter outputs by mean and standard deviation of last $\tau$ seconds of data
- How many standard deviations above the recent mean is this event registering?
- [GW150914: First results from the search for binary black hole coalescence with Advanced LIGO](https://dcc.ligo.org/public/0123/P1500269/016/LIGO-P1500269_GW150914_CBC_Search.pdf) used fixed windows of 2048s which were used for the next 2048s

Let's start with 30s for whatever reason (also going to reign in the bins since I'm psychic)

In [28]:
counts, bin_centers, Tb = build_background(
    min_value=-10, max_value=10, num_bins=1000, num_proc=8, max_tb=max_tb, norm_seconds=30
)

Output()

In [29]:
plot_background(counts, bin_centers, Tb)

And how do our events look with this method?

In [30]:
T, events = analyze_events()
plot_time_series(*events[event_names[0]])

In [31]:
plot_time_series(*events[event_names[1]])

In [32]:
plot_time_series(*events[event_names[2]])

In [33]:
plot_time_series(*events[event_names[3]])

And finally our FAR vs latency curves?

In [34]:
plot_far_vs_latency(events, counts, bin_centers, Tb)

Those $10^{-6}$ values in the plot above are really 0's, so for each event we're able to reach a matched filter output value that wasn't observed in the background distribution within 0.6 of the event registering.
- Background is flawed, and this is a small sample, but this should be the right framework for analyzing things

Next steps
- 